In [0]:
!pip install geopy # Подгрузка библиотеки геокодирования

     |████████████████████████████████| 125 kB 21.8 MB/s 
     |████████████████████████████████| 40 kB 1.3 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-92e1f92b-1f09-4e62-976e-bf72cd9276bf/bin/python -m pip install --upgrade pip' command.


In [0]:
from functools import partial
from geopy.geocoders import Nominatim
import pandas as pd
from geopy.extra.rate_limiter import RateLimiter
from geopy import distance
import re
import math

In [0]:
# Loading data
data = spark.read.option("sep", ";").option("header", True).csv("abfss://output@marsanalyticsdevadls.dfs.core.windows.net/RUSSIA_MW_DATAROOM/Coordinates_check/geo_check.csv")   
data_df = data.toPandas()
data_df

,OL_id,OLTradingName,OLDeliveryAddress,Latitude,Longitude
0,"1,0292E+14",Пятерочка,"г Москва, аллея 9-я Чоботовская, 1 корп. 1","55,655513","37,3523"
1,"1,0297E+14",Пятерочка №2559,"г Москва, аллея Беговая, 3","55,783296","37,564859"
2,"1,0167E+14",Верный 1071 (Союз Св. Иоанна Воина),"г Москва, аллея Беговая, 7Б","55,781403","37,564832"
3,"1,0005E+14",АО РН-Москва (РН-Москва АЗС№5),"г Москва, аллея Берёзовая, 12","55,84896952","37,61841595"
4,"1,0293E+14",красное и белое,"г Москва, аллея Берёзовая, 17 корп. 1","55,850069","37,6177788"
...,...,...,...,...,...
25730,"1,0097E+14","ООО Евроойл (пересечения а/д Омск-Новосибирск,...","Новосибирская, Чановский","55,2208549","75,992306"
25731,"1,0097E+14","ООО Евроойл (150км к югу от ж/ж переезда,п.Озе...","Новосибирская, Чановский, Озеро-Карачи, Вокзал...","55,33617401","76,96767426"
25732,"1,0005E+14",Водолага Л. Д ИП,"Новосибирская, Черепановский, Пятилетка, Центр...","54,239926","83,324564"
25733,"1,0097E+14",Сиб Ойл ЛТД,"Новосибирская, Черепановский, Южный, Школьная, 21","54,010485","83,422875"


In [0]:
def is_roman_number(num):                         # Обработка римских чисел
  pattern = re.compile(r"""   
                              ^M{0,3}
                              (CM|CD|D?C{0,3})?
                              (XC|XL|L?X{0,3})?
                              (IX|IV|V?I{0,3})?$
          """, re.VERBOSE)
  if re.match(pattern, num):
      return True
  return False
  
def romanToInt(s):
  result = 0
  comp = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
  i = 0
  while i < len(s) - 1:
      if comp[s[i+1]] > comp[s[i]]:
          result += comp[s[i+1]] - comp[s[i]]
          i += 2
      else:
          result += comp[s[i]]
          i += 1
  if i < len(s):
      result += comp[s[-1]]
  return str(result)

In [0]:
def prepare_addresses(address):                             # Обработка входящего адреса
  norm_address = ''
  if address[-1] == ',':
    address = address[:-1]
  address = address.replace('б-р', 'бульвар')
  address = address.replace('р-н', 'район')
  address = address.replace('корп. ', 'к')
  address = address.replace('кв-л', 'квартал')
  address = address.replace('кв.', '')
  address = address.replace('им.', '')
  address = address.replace('стр.', 'с')
  address = address.replace(' стр ', ' c')
  address = address.replace('строение ', 'с')
  address = address.split('место')[0]
  address = address.replace('место влад ', '')
  address = address.replace('место вл.', '')
  address = address.replace('место ', '')
  address = address.replace('павильон', '')
  address = address.replace('пр-кт', 'проспект')
  address = address.replace('тер.', '')
  parts_address = address.split(',')
  new_parts = []
  for part in parts_address:
    sp = part.split()
    for i, j in enumerate(sp):
#      if sp[i].isdigit() == True and sp[i-1].isdigit() == True:   #здесь ошибка из-за которой не читаются числа
#        sp.pop(i)
      if is_roman_number(j) == True:
        sp[i] = romanToInt(sp[i])
      if i >= 1:
        if sp[i-1] == 'Авиапарк':
          sp.pop(i)
        if '/' in sp[i-1]:
          sp.pop(i)
      if j == 'рп':
        sp[i] = 'городское поселение'
      if j == 'п':
        sp[i] = 'поселение'
      if j == 'пл':
        sp[i] = 'площадь'
      if j == 'ш':
        sp[i] = 'шоссе'
      if j == 'дп':
        sp.pop(i)
      if j == 'д':
        sp.pop(i)
      if j == 'г':
        sp.remove('г')
      if j == 'обл':
        sp[i] = 'область'
      if j == 'с':
        sp[i] = 'село'
      if j == 'ул':
        sp[i] = 'улица'
      if j == 'мкр':
        sp[i] = 'микрорайон'
      if j == 'аллея':
        sp.remove('аллея')
        sp.append('аллея')
      if j == 'км':
        sp.remove('км')
        sp.append('километр')
    new_parts.append(' '.join(sp))
  сh_address = ', '.join(new_parts)
  if сh_address.split(",")[0][-2:] == "ая":
    parts = сh_address.split(", ")
    parts[0] = parts[0] + " обл"
    сh_address = ', '.join(parts)
    print(сh_address)
  elif сh_address.split(",")[0] == "Москва":
    parts = сh_address.split(", ")
    parts[0] = parts[0] + ", Москва"
    сh_address = ', '.join(parts)
  elif сh_address.split(",")[0] == "Санкт-Петербург":
    parts = сh_address.split(", ")
    parts[0] = parts[0] + ", Санкт-Петербург"
    сh_address = ', '.join(parts)
  parts_address = сh_address.split()
  norm_address = ' '.join(parts_address)
  return norm_address

In [0]:
def find_full_address(address, geocode):  # Получение координат и ссылки на адрес в картах
  ar0 = []
  ar1 = []
  ar2 = []
  ar3 = []
  for i in range(address.shape[0]):
    norm_address = prepare_addresses(address[i])
    location = geocode(norm_address)
    if location != None:
      print('#{} processed'.format(i))
      ar0.append(location)
      ar1.append('https://www.openstreetmap.org/search?whereami=1&query={0}%2C{1}#map=16/{0}/{1}'.format(location.latitude, location.longitude))
      ar2.append(location.latitude)
      ar3.append(location.longitude)
    else:
      print('#{} not processed'.format(i))
      ar0.append(0)
      ar1.append(0)
      ar2.append(0)
      ar3.append(0)
  return ar0, ar1, ar2, ar3

def find_latitude(address):
  geolocator = Nominatim(user_agent='Tester')
  norm_address = prepare_addresses(address)
  location = geolocator.geocode(norm_address)
  return location.latitude

def find_longitude(address):
  geolocator = Nominatim(user_agent='Tester')
  norm_address = prepare_addresses(address)
  location = geolocator.geocode(norm_address)
  return location.longitude

In [0]:
def find_distance(lat1, lon1, address, lat2, lon2):   # Поиск расстояния между исходными координатами и найденными
  arr_coords = []
  for i in range(len(lat1)):
    if address[i] != '0':
      mat1 = math.cos(math.radians(90-lat1[i]))*math.cos(math.radians(90-lat2[i]))+math.sin(math.radians(90-lat1[i]))*math.sin(math.radians(90-lat2[i]))*math.cos(math.radians(lon1[i]-lon2[i]))
      if mat1 >= 1:
        mat1 = 1
      arr_coords.append(math.acos(mat1)*6371*1000)
    else:
      arr_coords.append('None')
  return arr_coords

In [0]:
geolocator = Nominatim(user_agent='coords_check')                              # Геокодирование
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
#address = prepare_addresses(data_df['OLDeliveryAddress'][0])
#partial(geocode(address), language='ru')
#normal_addresses = data_df['OLDeliveryAddress'].apply(lambda x: prepare_addresses(x))
#normal_addresses = normal_addresses.tolist()
arr0, arr1, arr2, arr3 = find_full_address(data_df['OLDeliveryAddress'], geocode)
data_df['Latitude'] = data_df['Latitude'].apply(lambda x: x.replace(',', '.'))
data_df['Longitude'] = data_df['Longitude'].apply(lambda x: x.replace(',', '.'))
#data_df['Check_latitude'] = data_df['OLDeliveryAddress'].apply(lambda x: find_latitude(x))
#data_df['Check_longitude'] = data_df['OLDeliveryAddress'].apply(lambda x: find_longitude(x))

#0 processed
#1 processed
#2 processed
#3 processed
#4 processed
#5 processed
#6 processed
#7 processed
#8 processed
#9 processed
#10 processed
#11 processed
#12 processed
#13 processed
#14 processed
#15 processed
#16 processed
#17 processed
#18 processed
#19 processed
#20 processed
#21 processed
#22 processed
#23 processed
#24 processed
#25 processed
#26 processed
#27 processed
#28 processed
#29 processed
#30 processed
#31 processed
#32 processed
#33 processed
#34 processed
#35 processed
#36 processed
#37 processed
#38 processed
#39 processed
#40 processed
#41 processed
#42 processed
#43 processed
#44 processed
#45 processed
#46 processed
#47 processed
#48 processed
#49 processed
#50 processed
#51 processed
#52 processed
#53 processed
#54 processed
#55 processed
#56 processed
#57 processed
#58 processed
#59 processed
#60 processed
#61 processed
#62 processed
#63 processed
#64 processed
#65 processed
#66 processed
#67 processed
#68 processed
#69 processed
#70 processed
#71 processed
#7

In [0]:
res0 = [i for i in arr0 if i != None]    # Очистка
res1 = []
for i in res0:
  if i != 0:
    res1.append(i.address)
  else:
    res1.append('0')
result0 = [i for i in arr1 if i != None]
result1 = []
for i in result0:
  if i != 0:
    result1.append(i)
  else:
    result1.append('0')
result2 = [i for i in arr2 if i != None]
result3 = [i for i in arr3 if i != None]

In [0]:
print(len(res1), len(result1), len(result2), len(result3))

25735 25735 25735 25735


In [0]:
data_df['URL'] = result1
data_df['Address_check'] = res1
data_df['Latitude_check'] = result2
data_df['Longitude_check'] = result3

In [0]:
data_df['Latitude'] = data_df['Latitude'].astype(float)
data_df['Longitude'] = data_df['Longitude'].astype(float)
data_df['Latitude_check'] = data_df['Latitude_check'].astype(float)
data_df['Longitude_check'] = data_df['Longitude_check'].astype(float)

In [0]:
data_df['Distance'] = find_distance(data_df['Latitude'], data_df['Longitude'], data_df['Address_check'], data_df['Latitude_check'], data_df['Longitude_check'])

In [0]:
data_df['Distance'] = data_df['Distance'].astype(str)

In [0]:
spark_df = spark.createDataFrame(data_df)
spark_df.coalesce(1).write.mode("overwrite").option("encoding", "UTF-8").option("header", "true").option("delimiter",";").csv("dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/results")

In [0]:
# Проверка
dd = spark.read.option("sep", ";").option("header", True).csv("abfss://output@marsanalyticsdevadls.dfs.core.windows.net/RUSSIA_MW_DATAROOM/Coordinates_check/results/part-00000-tid-42175458780702472-db8ee672-bddd-4929-982a-defb2a44f7eb-25024-1-c000.csv")
dd_df = dd.toPandas()
dd_df

,OL_id,OLTradingName,OLDeliveryAddress,Latitude,Longitude,URL,Address_check,Latitude_check,Longitude_check,Distance
0,"1,0292E+14",Пятерочка,"г Москва, аллея 9-я Чоботовская, 1 корп. 1",55.655513,37.3523,https://www.openstreetmap.org/search?whereami=...,"9-я Чоботовская аллея, Ново-Переделкино, Москв...",55.6553594,37.3531791,57.732526856152354
1,"1,0297E+14",Пятерочка №2559,"г Москва, аллея Беговая, 3",55.783296,37.564859,https://www.openstreetmap.org/search?whereami=...,"УПП 43, 3, Беговая аллея, район Беговой, Москв...",55.7828872,37.5651848,49.8125427009244
2,"1,0167E+14",Верный 1071 (Союз Св. Иоанна Воина),"г Москва, аллея Беговая, 7Б",55.781403,37.564832,https://www.openstreetmap.org/search?whereami=...,"Верный, 7Б, Беговая аллея, район Беговой, Моск...",55.781404949999995,37.56484487672087,0.8384458494093232
3,"1,0005E+14",АО РН-Москва (РН-Москва АЗС№5),"г Москва, аллея Берёзовая, 12",55.84896952,37.61841595,https://www.openstreetmap.org/search?whereami=...,"12, Берёзовая аллея, район Отрадное, Москва, Ц...",55.85424145,37.60527522242275,1008.1671399272203
4,"1,0293E+14",красное и белое,"г Москва, аллея Берёзовая, 17 корп. 1",55.850069,37.6177788,https://www.openstreetmap.org/search?whereami=...,"17 к1, Берёзовая аллея, Останкинский район, Мо...",55.850569449999995,37.618772423697976,83.32691266881069
...,...,...,...,...,...,...,...,...,...,...
25730,"1,0097E+14","ООО Евроойл (пересечения а/д Омск-Новосибирск,...","Новосибирская, Чановский",55.2208549,75.992306,https://www.openstreetmap.org/search?whereami=...,"Чановский район, Новосибирская область, Сибирс...",55.1620629,76.85672511415562,55255.90965466575
25731,"1,0097E+14","ООО Евроойл (150км к югу от ж/ж переезда,п.Озе...","Новосибирская, Чановский, Озеро-Карачи, Вокзал...",55.33617401,76.96767426,https://www.openstreetmap.org/search?whereami=...,"Вокзальная улица, Озеро-Карачи, Озеро-Карачинс...",55.3368722,76.9539047,874.2780506370036
25732,"1,0005E+14",Водолага Л. Д ИП,"Новосибирская, Черепановский, Пятилетка, Центр...",54.239926,83.324564,https://www.openstreetmap.org/search?whereami=...,"Центральная улица, Пятилетка, Пятилетский сель...",54.2411243,83.3275276,234.1790376282739
25733,"1,0097E+14",Сиб Ойл ЛТД,"Новосибирская, Черепановский, Южный, Школьная, 21",54.010485,83.422875,https://www.openstreetmap.org/search?whereami=...,"Школьная улица, Южный, Безменовский сельсовет,...",54.0103432,83.4222948,41.059818186679536


In [0]:
# Проверка
dd_df['Latitude'] = dd_df['Latitude'].astype(float)
dd_df['Longitude'] = dd_df['Longitude'].astype(float)
dd_df['Latitude_check'] = dd_df['Latitude_check'].astype(float)
dd_df['Longitude_check'] = dd_df['Longitude_check'].astype(float)
dd_df['Distance'] = dd_df['Distance'].replace('None', 1000)
dd_df['Distance'] = dd_df['Distance'].astype(float)

In [0]:
unproc_df = dd_df[dd_df["Distance"] > 300].reset_index(drop=True)

##Отсюда запускать корректировки

In [0]:
%fs ls /mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data

path,name,size,modificationTime
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data/_SUCCESS,_SUCCESS,0,1697098535000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data/_committed_6613145131748893222,_committed_6613145131748893222,114,1697098534000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data/_started_6613145131748893222,_started_6613145131748893222,0,1697098533000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data/part-00000-tid-6613145131748893222-90252e93-e063-472f-89c8-a4e9bed45384-9955-1-c000.csv,part-00000-tid-6613145131748893222-90252e93-e063-472f-89c8-a4e9bed45384-9955-1-c000.csv,3037272,1697098534000


In [0]:
unproc_spark = spark.read.option("sep", ";").option("header", True).csv("abfss://output@marsanalyticsdevadls.dfs.core.windows.net/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data/part-00000-tid-6613145131748893222-90252e93-e063-472f-89c8-a4e9bed45384-9955-1-c000.csv")
unproc_df = unproc_spark.toPandas()

In [0]:
unproc_df.shape

Out[24]: (7433, 10)

In [0]:
from geopy.geocoders import Yandex  # Яндекс геокодер
import requests

def fetch_coordinates(apikey, address):
    base_url = "https://geocode-maps.yandex.ru/1.x"
    response = requests.get(base_url, params={
        "geocode": address,
        "apikey": apikey,
        "format": "json",
    })
    response.raise_for_status()
    found_places = response.json()['response']['GeoObjectCollection']['featureMember']

    if not found_places:
        return None

    most_relevant = found_places[0]
    lon, lat = most_relevant['GeoObject']['Point']['pos'].split(" ")
    address = most_relevant['GeoObject']['metaDataProperty']['GeocoderMetaData']['text']
    return lat, lon, address

In [0]:
def find_dist_yandex(lat1, lon1, lat2, lon2):
  distance = 0
  mat1 = math.cos(math.radians(90-lat1))*math.cos(math.radians(90-lat2))+math.sin(math.radians(90-lat1))*math.sin(math.radians(90-lat2))*math.cos(math.radians(lon1-lon2))
  if mat1 >= 1:
    mat1 = 1
  distance = math.acos(mat1)*6371*1000
  return distance

In [0]:
api_key = '06470783-9b79-4e9f-b4d1-9fca20cbf3d8'    # Необходимо указать свой API ключ

In [0]:
distances = []
num_requests = 980
for i in range(num_requests):
  address = fetch_coordinates(apikey=api_key, address = unproc_df['OLDeliveryAddress'][i])
  distances.append(find_dist_yandex(float(unproc_df['Latitude'][i]), float(unproc_df['Longitude'][i]), float(address[0]), float(address[1])))

del address

In [0]:
processed_data = unproc_df[:980].copy()   # заменить на num_requests
unprocessed_data = unproc_df[980:].copy().reset_index(drop=True) # заменить на num_requests
processed_data['Distance'] = distances

In [0]:
%fs ls /mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data

path,name,size,modificationTime
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/_committed_3016345693170384283,_committed_3016345693170384283,214,1697098532000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/_committed_5337368393145993664,_committed_5337368393145993664,114,1697098314000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/_committed_661616498384240398,_committed_661616498384240398,203,1697447755000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/_committed_vacuum3309538041368066216,_committed_vacuum3309538041368066216,129,1697447756000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/_started_661616498384240398,_started_661616498384240398,0,1697447754000
dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/part-00000-tid-661616498384240398-3d7f6ab9-aec4-4b2c-9695-2ae595349faa-32486-1-c000.csv,part-00000-tid-661616498384240398-3d7f6ab9-aec4-4b2c-9695-2ae595349faa-32486-1-c000.csv,439741,1697447755000


In [0]:
preprocdata = spark.read.option("sep", ";").option("header", True).csv("abfss://output@marsanalyticsdevadls.dfs.core.windows.net/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data/part-00000-tid-661616498384240398-3d7f6ab9-aec4-4b2c-9695-2ae595349faa-32486-1-c000.csv")

In [0]:
new_preprocdata = pd.concat([preprocdata, processed_data])

In [0]:
spark_processed_data = spark.createDataFrame(new_preprocdata)
spark_processed_data.coalesce(1).write.mode("overwrite").option("encoding", "UTF-8").option("header", "true").option("delimiter",";").csv("dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/pre_processed_data")
spark_unprocessed_data = spark.createDataFrame(unprocessed_data)
spark_unprocessed_data.coalesce(1).write.mode("overwrite").option("encoding", "UTF-8").option("header", "true").option("delimiter",";").csv("dbfs:/mnt/DEV/OUTPUT/RUSSIA_MW_DATAROOM/Coordinates_check/unprocessed_data")

##Блок с проверками

In [0]:
# Для проверок
find_dist_yandex(unproc_df['Latitude'][1], unproc_df['Longitude'][1], float(coords[0]), float(coords[1]))

Out[19]: 125.4089177506201

In [0]:
# Для проверок
geolocator = Nominatim(user_agent='Tester')
#adress = 'Россия, Московская область, городской округ Домодедово, аэропорт Домодедово имени М.В. Ломоносова, 1'
#noad = prepare_addresses(adress)
#print(noad)
#location = geolocator.geocode(noad)
loc = geolocator.reverse('55.802167, 37.300347')
#index = location.address.split(',')[-2]
#print(location)
#print(index)
print(loc)
#print(location.latitude, location.longitude)

Глобус, вл1 с1, Новорижское шоссе, Райцентр, городской округ Красногорск, Московская область, Центральный федеральный округ, 143405, Россия


In [0]:
# Для проверок
find_distance([55.848969], [37.618415], ['something'], [55.854241], [37.605275])

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1589131758457307> in <cell line: 2>()
      1 # Для проверок
----> 2 find_distance([55.848969], [37.618415], ['something'], [55.854241], [37.605275])

NameError: name 'find_distance' is not defined

In [0]:
print('It works')

It works


In [0]:
data = spark.read.option("sep", ";").option("header", True).csv("abfss://output@marsanalyticsdevadls.dfs.core.windows.net/RUSSIA_MW_DATAROOM/Coordinates_check/full_coords.csv")
orig = spark.read.option("sep", ";").option("header", True).csv("abfss://output@marsanalyticsdevadls.dfs.core.windows.net/RUSSIA_MW_DATAROOM/Coordinates_check/geo_check.csv")

In [0]:
df = data.toPandas()
dff = orig.toPandas()

In [0]:
df

,id,title,geopos,address,postal_code,city,region
0,166427044398,Фуд Сити,"{'type': 'Point', 'coordinates': [37.466404, 5...","108814, Россия, Москва, поселение Сосенское, К...",108814,г. Москва,Москва
1,1086565414,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.545225, 5...","142181, Россия, Московская область, Подольск, ...",142181,г. Подольск,Московская область
2,1330374048,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.300347, 5...","121500, Россия, Москва, Новорижское шоссе, 22-...",121500,г. Москва,Москва
3,190278434259,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.885653, 5...","140006, Россия, Московская область, Котельники...",140006,г. Котельники,Московская область
4,1506823238,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.886264, 5...","141202, Россия, Московская область, Пушкино, К...",141202,г. Пушкино,Московская область
...,...,...,...,...,...,...,...
862706,1975990001,Поворот на Посьет,"{'type': 'Point', 'coordinates': [130.83553290...",None,692715,None,Приморский край
862707,1975990141,Поворот на Посьет,"{'type': 'Point', 'coordinates': [130.83120153...",None,692715,None,Приморский край
862708,4433003940,Гвоздево,"{'type': 'Point', 'coordinates': [130.916472, ...",None,692715,None,Приморский край
862709,1543342815,Посьет-3,"{'type': 'Point', 'coordinates': [130.80580161...",None,692715,пгт Посьет,Приморский край


In [0]:
dff

,OL_id,OLTradingName,OLDeliveryAddress,Latitude,Longitude
0,"1,0292E+14",Пятерочка,"г Москва, аллея 9-я Чоботовская, 1 корп. 1","55,655513","37,3523"
1,"1,0297E+14",Пятерочка №2559,"г Москва, аллея Беговая, 3","55,783296","37,564859"
2,"1,0167E+14",Верный 1071 (Союз Св. Иоанна Воина),"г Москва, аллея Беговая, 7Б","55,781403","37,564832"
3,"1,0005E+14",АО РН-Москва (РН-Москва АЗС№5),"г Москва, аллея Берёзовая, 12","55,84896952","37,61841595"
4,"1,0293E+14",красное и белое,"г Москва, аллея Берёзовая, 17 корп. 1","55,850069","37,6177788"
...,...,...,...,...,...
25730,"1,0097E+14","ООО Евроойл (пересечения а/д Омск-Новосибирск,...","Новосибирская, Чановский","55,2208549","75,992306"
25731,"1,0097E+14","ООО Евроойл (150км к югу от ж/ж переезда,п.Озе...","Новосибирская, Чановский, Озеро-Карачи, Вокзал...","55,33617401","76,96767426"
25732,"1,0005E+14",Водолага Л. Д ИП,"Новосибирская, Черепановский, Пятилетка, Центр...","54,239926","83,324564"
25733,"1,0097E+14",Сиб Ойл ЛТД,"Новосибирская, Черепановский, Южный, Школьная, 21","54,010485","83,422875"


In [0]:
prepare_addresses(dff['OLDeliveryAddress'][0])

Out[29]: 'Москва, Москва, 9-я Чоботовская аллея, 1 к1'

In [0]:
dff['OLDeliveryAddress'][25730].split(", ")[0] + " обл"

Out[25]: 'Новосибирская обл'

In [0]:
df['address'][2].split(", ")[2:]

Out[35]: ['Москва', 'Новорижское шоссе', '22-й километр', 'вл1с1']

In [0]:
df = df.fillna("0")

In [0]:
def normalize_adres(dataframe):
  dataframe['norm_adres'] = dataframe['address'].apply(lambda x: ', '.join(x.split(", ")[2:]))
  new_adr = []
  for i in range(dataframe.shape[0]):
    parts = dataframe['norm_adres'][i].split(", ")
    if parts[0] == "Москва":
      parts[0] = parts[0] + ", Москва"
      new_adr.append(', '.join(parts))
    elif parts[0] == "Санкт-Петербург":
      parts[0] = parts[0] + ", Санкт-Петербург"
      new_adr.append(', '.join(parts))
    else:
      new_adr.append(', '.join(parts))
  dataframe['new_adr'] = new_adr
  return dataframe

In [0]:
df = normalize_adres(df)

In [0]:
df.head(15)

,id,title,geopos,address,postal_code,city,region,norm_adres,new_adr
0,166427044398,Фуд Сити,"{'type': 'Point', 'coordinates': [37.466404, 5...","108814, Россия, Москва, поселение Сосенское, К...",108814,г. Москва,Москва,"Москва, поселение Сосенское, Калужское шоссе, ...","Москва, Москва, поселение Сосенское, Калужское..."
1,1086565414,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.545225, 5...","142181, Россия, Московская область, Подольск, ...",142181,г. Подольск,Московская область,"Московская область, Подольск, микрорайон Климо...","Московская область, Подольск, микрорайон Климо..."
2,1330374048,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.300347, 5...","121500, Россия, Москва, Новорижское шоссе, 22-...",121500,г. Москва,Москва,"Москва, Новорижское шоссе, 22-й километр, вл1с1","Москва, Москва, Новорижское шоссе, 22-й киломе..."
3,190278434259,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.885653, 5...","140006, Россия, Московская область, Котельники...",140006,г. Котельники,Московская область,"Московская область, Котельники, Новорязанское ...","Московская область, Котельники, Новорязанское ..."
4,1506823238,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.886264, 5...","141202, Россия, Московская область, Пушкино, К...",141202,г. Пушкино,Московская область,"Московская область, Пушкино, Красноармейское ш...","Московская область, Пушкино, Красноармейское ш..."
5,169890191777,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.944601, 5...","143912, Россия, Московская область, Балашиха, ...",143912,г. Балашиха,Московская область,"Московская область, Балашиха, шоссе Энтузиасто...","Московская область, Балашиха, шоссе Энтузиасто..."
6,129498155424,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.420997, 5...","108811, Россия, Москва, поселение Московский, ...",108811,г. Москва,Москва,"Москва, поселение Московский, Киевское шоссе, ...","Москва, Москва, поселение Московский, Киевское..."
7,1764358261,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.177869, 5...","143081, Россия, Московская область, Одинцовски...",143081,0,Московская область,"Московская область, Одинцовский городской окру...","Московская область, Одинцовский городской окру..."
8,1071756074,Глобус гипермаркет,"{'type': 'Point', 'coordinates': [37.846075, 5...","141077, Россия, Московская область, Королёв, К...",141077,г. Королев,Московская область,"Московская область, Королёв, Коммунальная улиц...","Московская область, Королёв, Коммунальная улиц..."
9,10787552670,Булочная Ф. Вольчека,"{'type': 'Point', 'coordinates': [30.348261, 5...","191025, Россия, Санкт-Петербург, Литейный прос...",191025,г. Санкт-Петербург,Санкт-Петербург,"Санкт-Петербург, Литейный проспект, 64/78","Санкт-Петербург, Санкт-Петербург, Литейный про..."
